# 1NN против RandomForest

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.



## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании.

In [1]:
from sklearn.datasets import load_digits
from scipy.spatial import distance
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
data_digits = load_digits()

In [3]:
data_digits.keys()

['images', 'data', 'target_names', 'DESCR', 'target']

In [4]:
data_digits.data.shape

(1797, 64)

In [5]:
X_train = data_digits.data[:int(data_digits.data.shape[0]*0.75),:]
X_test = data_digits.data[int(data_digits.data.shape[0]*0.75):,:]
y_train = data_digits.target[:int(data_digits.target.shape[0]*0.75)]
y_test = data_digits.target[int(data_digits.data.shape[0]*0.75):]

In [12]:
euclid_list = []
num_clas = []
ans_list = []
ans = np.ones(450)
k= 0
for element1 in X_test:
    i = 0
    for element2 in X_train:
        euclid_list.append(distance.euclidean(element2,element1))
        num_clas.append(y_train[i])
        i+=1
    index = euclid_list.index(min(euclid_list))
    ans[k] = num_clas[index]
    euclid_list = []
    num_clas = []
    k+=1

    

In [13]:
x = 0
for i in range(y_test.shape[0]):
    if ans[i] == y_test[i]:
        x+=1
print 1 - float(x)/y_test.shape[0] 
        

0.0377777777778


# Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [232]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [234]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, clf.predict(X_test))
print 1 - score

0.0733333333333
